In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv("archive/data.csv", names = ['label','id', 'time', 'flag', 'user','text'],header=None)

In [3]:
df.head(10)

,label,id,time,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [4]:
df['label'].unique()

array([0, 4], dtype=int64)

In [5]:
df['label'] = df['label'].map(lambda x: 1 if( x == 4) else 0 )

df.tail(5)

,label,id,time,flag,user,text
1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,1,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [6]:
df.drop(["id",'time','flag','user'],axis=1,inplace = True)

df.head(5)

,label,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [7]:

df['text'] = df['text'].replace(
        to_replace ='@\w+', value = '', regex = True
    ).replace(
        to_replace ='https?://\S+', value = '', regex = True
    ).replace(
        to_replace ='www\.\S+', value = '', regex = True
    ).replace(
        to_replace= '[^\w\s]', value='', regex = True
    ).str.lower()

df.head(10)

,label,text
0,0,awww thats a bummer you shoulda got david ...
1,0,is upset that he cant update his facebook by t...
2,0,i dived many times for the ball managed to sa...
3,0,my whole body feels itchy and like its on fire
4,0,no its not behaving at all im mad why am i he...
5,0,not the whole crew
6,0,need a hug
7,0,hey long time no see yes rains a bit only a ...
8,0,nope they didnt have it
9,0,que me muera


In [8]:
total_samples = 1600000
classes = 2
no_of_samples_per_class = 800000
mean = df['text'].map(lambda x: len(x.split())).mean()
mean

12.49858625

In [9]:
len(max(df['text'], key=len))

180

In [10]:
shuffled_df = df.reindex(np.random.permutation(df.index))

shuffled_df

,label,text
1233249,1,coming to new orleans anytime soon
811724,1,packing for holidays only 3 days omg
718547,0,just finished watching selena the movie so sad
455062,0,so fucking mad agh i hate being 13 and having ...
963780,1,going to walmart with my mummy she is alll go...
...,...,...
1379690,1,please support electrik red and sign these ...
349863,0,whats got me insane cause i cant stand the rai...
518859,0,gmail still down here
477545,0,hey sweetiemy weekend was sad went to a funer...


In [11]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(shuffled_df['text'], shuffled_df['label'], random_state=0)

x_train.shape, x_val.shape, y_train.shape, y_val.shape

((1200000,), (400000,), (1200000,), (400000,))

In [12]:
from tensorflow.python.keras.preprocessing import text,sequence

top_k = 20000
max_sequence_length = 500

def sequence_vectorize(train_text, val_text):
    tokenizer = text.Tokenizer(num_words=top_k)
    tokenizer.fit_on_texts(train_text)
    
    x_train = tokenizer.texts_to_sequences(train_text)
    x_val = tokenizer.texts_to_sequences(val_text)
    
    max_length = len(max(x_train, key=len))
    
    if max_length > max_sequence_length:
        max_length = max_sequence_length
    
    x_train = sequence.pad_sequences(x_train, maxlen=max_length)
    x_val = sequence.pad_sequences(x_val, maxlen=max_length)
    
    return x_train, x_val, tokenizer.word_index

In [13]:
x_train, x_val, word_tokens = sequence_vectorize(x_train, x_val)

In [14]:
x_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,  453, 5027,   13,
        411,  106, 1204,    5, 3088,  691])

In [15]:
x_train.shape, x_val.shape

((1200000, 39), (400000, 39))

In [16]:
word_tokens

{'i': 1,
 'to': 2,
 'the': 3,
 'a': 4,
 'my': 5,
 'and': 6,
 'you': 7,
 'is': 8,
 'it': 9,
 'for': 10,
 'in': 11,
 'of': 12,
 'im': 13,
 'on': 14,
 'me': 15,
 'so': 16,
 'have': 17,
 'that': 18,
 'but': 19,
 'just': 20,
 'with': 21,
 'be': 22,
 'at': 23,
 'its': 24,
 'not': 25,
 'was': 26,
 'this': 27,
 'now': 28,
 'good': 29,
 'up': 30,
 'day': 31,
 'get': 32,
 'all': 33,
 'out': 34,
 'like': 35,
 'are': 36,
 'no': 37,
 'go': 38,
 'dont': 39,
 'do': 40,
 'your': 41,
 'today': 42,
 'going': 43,
 'too': 44,
 'love': 45,
 'cant': 46,
 'work': 47,
 'got': 48,
 'time': 49,
 'back': 50,
 'from': 51,
 'lol': 52,
 'u': 53,
 'one': 54,
 'what': 55,
 'will': 56,
 'know': 57,
 'we': 58,
 'about': 59,
 'really': 60,
 'am': 61,
 'had': 62,
 'can': 63,
 'see': 64,
 'some': 65,
 'well': 66,
 'if': 67,
 'still': 68,
 'new': 69,
 'want': 70,
 'how': 71,
 'think': 72,
 'night': 73,
 'amp': 74,
 'thanks': 75,
 'as': 76,
 'home': 77,
 'when': 78,
 'there': 79,
 'oh': 80,
 '2': 81,
 'more': 82,
 'much': 8

In [36]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

def sepcnn_model(blocks,
                 filters,
                 kernel_size,
                 embedding_dim,
                 dropout_rate,
                 pool_size,
                 input_shape,
                 num_features):
    model = Sequential()
    
    model.add(layers.Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=input_shape[0],
                              name= 'embedding'))
    for _ in range(blocks-1):
        model.add(layers.Dropout(rate=dropout_rate))
        model.add(layers.SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        model.add(layers.MaxPooling1D(pool_size=pool_size))
    model.add(layers.Dropout(rate=dropout_rate))
    model.add(layers.SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(layers.GlobalAveragePooling1D())
    model.add(layers.Dropout(rate=dropout_rate))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model



In [37]:
mymodel = sepcnn_model(blocks=2,
                 filters= 32,
                 kernel_size = 3,
                 embedding_dim = 200,
                 dropout_rate = 0.4,
                 pool_size = 2,
                 input_shape = (x_train.shape[1],1),
                 num_features = 20000)

mymodel.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.01),
               loss = 'binary_crossentropy',
               metrics = ['binary_accuracy'])
mymodel.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 39, 200)           4000000   
                                                                 
 dropout_21 (Dropout)        (None, 39, 200)           0         
                                                                 
 separable_conv1d_40 (Separa  (None, 39, 32)           7032      
 bleConv1D)                                                      
                                                                 
 max_pooling1d_12 (MaxPoolin  (None, 19, 32)           0         
 g1D)                                                            
                                                                 
 dropout_22 (Dropout)        (None, 19, 32)            0         
                                                                 
 separable_conv1d_41 (Separa  (None, 19, 64)          

In [38]:
from tensorflow.keras import callbacks

callback = callbacks.EarlyStopping(patience = 5,
                                  min_delta = 0.001,
                                  restore_best_weights = True)


In [41]:
print("Training on GPU...") if tf.config.list_physical_devices('GPU') else print("Training on CPU...")

Training on CPU...


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [ ]:
history = mymodel.fit(x_train, y_train,
         validation_data = (x_val, y_val),
         batch_size = 1024,
         epochs = 10,
         callbacks = [callback])

Epoch 1/10
1172/1172 [==============================] - 255s 217ms/step - loss: 0.4466 - binary_accuracy: 0.8007 - val_loss: 0.4443 - val_binary_accuracy: 0.8072
Epoch 2/10
1172/1172 [==============================] - 194s 166ms/step - loss: 0.4293 - binary_accuracy: 0.8087 - val_loss: 0.4465 - val_binary_accuracy: 0.8064
Epoch 3/10
1172/1172 [==============================] - 210s 179ms/step - loss: 0.4185 - binary_accuracy: 0.8141 - val_loss: 0.4471 - val_binary_accuracy: 0.8058
Epoch 4/10
1172/1172 [==============================] - 226s 192ms/step - loss: 0.4117 - binary_accuracy: 0.8169 - val_loss: 0.4500 - val_binary_accuracy: 0.8039
Epoch 5/10
1172/1172 [==============================] - 219s 187ms/step - loss: 0.4067 - binary_accuracy: 0.8194 - val_loss: 0.4399 - val_binary_accuracy: 0.8086
Epoch 6/10
1172/1172 [==============================] - 222s 190ms/step - loss: 0.4025 - binary_accuracy: 0.8215 - val_loss: 0.4763 - val_binary_accuracy: 0.7756
Epoch 7/10
 930/1172 [======